In [ ]:
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from typing import Any, Tuple, List, Dict

In [ ]:
def load_image(image_path: str) -> np.ndarray:
    """
    Load an image from disk.
    Raises FileNotFoundError if the image cannot be loaded.
    """
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found or unable to load: {image_path}")
    return image


def load_segmentation_model(model_path: str, device: str = 'cpu') -> Any:
    """
    Load a segmentation model (e.g., Ultralytics YOLO) and move it to the specified device.
    """
    model = YOLO(model_path)
    try:
        model.to(device)
    except Exception:
        pass
    return model


def segment_image(
    image: np.ndarray,
    model: Any,
    object_type: str,
    class_mapping: Dict[str, int]
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Perform instance segmentation on the image, returning a binary mask for the specified object type
    and a combined foreground mask of all detected objects.
    """
    results = model.predict(image, verbose=False)
    h, w = image.shape[:2]
    combined_fg = np.zeros((h, w), dtype=np.uint8)
    object_mask = np.zeros((h, w), dtype=np.uint8)
    target_cls = class_mapping.get(object_type)

    for res in results:
        if hasattr(res, 'masks') and res.masks is not None:
            masks = res.masks.data.cpu().numpy()
            classes = res.boxes.cls.cpu().numpy().astype(int)
            for m, cls in zip(masks, classes):
                if m.shape != (h, w):
                    m = cv2.resize(m, (w, h), interpolation=cv2.INTER_NEAREST)
                mask_bin = (m > 0.5).astype(np.uint8) * 255
                combined_fg = cv2.bitwise_or(combined_fg, mask_bin)
                if cls == target_cls:
                    object_mask = cv2.bitwise_or(object_mask, mask_bin)

    bg_mask = cv2.bitwise_not(combined_fg)
    return object_mask, bg_mask


def get_distance_map(
    image_shape: Tuple[int, int],
    object_distance_info: Any
) -> np.ndarray:
    """
    Obtain or construct a per-pixel distance map.
    """
    h, w = image_shape
    if isinstance(object_distance_info, np.ndarray):
        if object_distance_info.shape != (h, w):
            raise ValueError("Distance array shape does not match image shape.")
        return object_distance_info.astype(float)
    if isinstance(object_distance_info, (float, int)):
        return np.full((h, w), float(object_distance_info), dtype=float)
    if isinstance(object_distance_info, str):
        dm = cv2.imread(object_distance_info, cv2.IMREAD_UNCHANGED)
        if dm is None:
            raise FileNotFoundError(f"Depth map not found: {object_distance_info}")
        dm = dm.astype(float)
        if dm.max() > 1:
            dm = dm / 255.0
        return dm
    raise TypeError("Unsupported object_distance_info type.")










## These are important functions that acctually blur the image##



In [ ]:
def calculate_sigma_map(
    distance_map: np.ndarray,
    blur_start: float,
    blur_growth_rate: float
) -> np.ndarray:
    """
    Compute a per-pixel sigma map based on distance. No blur if d <= blur_start,
    then linear growth beyond.
    """
    sigma_map = np.zeros_like(distance_map, dtype=float)
    mask = distance_map > blur_start
    sigma_map[mask] = (distance_map[mask] - blur_start) * blur_growth_rate
    return sigma_map




def apply_spatial_gaussian_blur(
    image: np.ndarray,
    sigma_map: np.ndarray,
    sigma_levels: List[float],
    ksizes: List[int]
) -> np.ndarray:
    """
    Approximate spatially varying Gaussian blur by quantizing sigma_map into discrete levels,
    blurring the image at each level, and compositing via masks.
    """
    if len(sigma_levels) != len(ksizes):
        raise ValueError("sigma_levels and ksizes must have the same length.")
    if sigma_levels[0] != 0 or ksizes[0] != 1:
        raise ValueError("First sigma level must be 0 with kernel size 1.")

    blurred_variants: Dict[float, np.ndarray] = {0.0: image.copy()}
    for s, k in zip(sigma_levels[1:], ksizes[1:]):
        k_safe = max(1, k if k % 2 == 1 else k + 1)
        blurred_variants[s] = cv2.GaussianBlur(image, (k_safe, k_safe), sigmaX=s)

    output = image.copy()
    for i in range(1, len(sigma_levels)):
        prev_s = sigma_levels[i-1]
        curr_s = sigma_levels[i]
        mask = (sigma_map > prev_s) & (sigma_map <= curr_s)
        output = np.where(mask[..., np.newaxis], blurred_variants[curr_s], output)
    max_s = sigma_levels[-1]
    mask_max = sigma_map > max_s
    if np.any(mask_max):
        output = np.where(mask_max[..., np.newaxis], blurred_variants[max_s], output)
    return output

In [ ]:
# Custom color tint for better blur visibility, it is called in the main area

def apply_color_tint(
    image: np.ndarray,
    mask: np.ndarray,
    color: Tuple[int, int, int],
    alpha: float = 0.3
) -> np.ndarray:
    """
    Apply a semi-transparent color tint to the masked regions of the image.
    mask: 2D boolean array where True indicates pixels to tint.
    color: BGR tuple, alpha: tint strength.
    """
    # 1) Make sure mask is boolean
    mask_bool = mask.astype(bool)

    # 2) Create an empty overlay and paint only the masked pixels with your tint color
    overlay = np.zeros_like(image, dtype=np.uint8)
    overlay[mask_bool] = color  # broadcast (h,w,3) ← (3,)

    # 3) Blend original + overlay (only the colored bits will actually change)
    blended = cv2.addWeighted(image, 1 - alpha, overlay, alpha, 0)

    # 4) Copy the blended (tinted) pixels back onto your original image
    output = image.copy()
    output[mask_bool] = blended[mask_bool]

    return output


## Custom Distance masks for manaul blur filter effect ##

Basically you need to fill an array with the desired distance values, the farther(greater) the distance the more blur. The map must be the same shape as the image.

In [1]:
def generate_heart_depth_map(
    image_shape: Tuple[int, int],
    min_depth: float = 0.0,
    max_depth: float = 1.0,
    heart_scale: float = 5
) -> np.ndarray:
    """
    Generate an artificial depth map with:
      - A smaller heart (scaled by `heart_scale`) at constant depth = min_depth
      - Everywhere else depth = min_depth + gradient increasing toward the edges,
        reaching max_depth at the furthest corners.
    """
    h, w = image_shape

    # 1) Build normalized coordinate grid centered at (0,0)
    ys = np.linspace(-1.5, 1.5, h)[:, None]
    xs = np.linspace(-1.0, 1.0, w)[None, :]

    # 2) Shrink coords for the heart mask
    xs_h = xs * heart_scale
    ys_h = ys * heart_scale

    # 3) Implicit heart equation (inside <=1)
    row_mask = (
        xs_h**2
        + (5.0/4.0 * ys_h - np.sqrt(np.abs(xs_h)))**2
        <= 1.0
    )

    # 4) Compute radial distance from center, normalized to [0, 1]
    #    max possible radius is at a corner: sqrt(1^2 + 1.5^2)
    max_radius = np.sqrt(1.0**2 + 1.5**2)
    radius = np.sqrt(xs**2 + ys**2)
    norm_radius = np.clip(radius / max_radius, 0.0, 1.0)

    # 5) Build the gradient: at center = min_depth, at edges = max_depth
    gradient = min_depth + norm_radius * (max_depth - min_depth)

    # 6) Combine: heart = flat min_depth; outside = gradient
    depth_map = np.where(row_mask, min_depth, gradient).astype(float)
    return depth_map

def generate_constant_rectangle_depth_map(
    image_shape: Tuple[int, int],
    min_depth: float = 0.0,
    max_depth: float = 50.0,
) -> np.ndarray:
    """
    Generate a depth map with three horizontal bands:
      - Top third:      depth = max_depth
      - Middle third:   depth = max_depth / 2
      - Bottom third:   depth = min_depth
    """
    h, w = image_shape
    # compute band boundaries (integer row indices)
    top_end = h // 3
    mid_end = (2 * h) // 3

    # initialize all to zero (or any placeholder)
    depth_map = np.empty((h, w), dtype=float)

    # fill each band
    depth_map[:top_end, :] = max_depth
    depth_map[top_end:mid_end, :] = max_depth / 10.0
    depth_map[mid_end:, :] = min_depth

    return depth_map

NameError: name 'Tuple' is not defined

## Main Function Area ##

Edit the arguments for the blur function

In [2]:
def main(
    image_path: str,
    object_distance_info: Any,
    object_type: str,
    config: Dict[str, Any],
    use_synthmap: bool = True
) -> None:
    """
    Orchestrate the spatially varying blur pipeline.
    """
    img = load_image(image_path)
    # Depth map selection
    if use_synthmap:
        #base_dist = generate_synthmap_depth_map(img.shape[:2])
        base_dist = generate_constant_rectangle_depth_map(img.shape[:2])
    else:
        base_dist = get_distance_map(img.shape[:2], object_distance_info)

    # Segmentation
    model = load_segmentation_model(config['model_path'], config.get('device', 'cpu'))
    object_mask, bg_mask = segment_image(img, model, object_type, config['class_mapping'])

    # Blur sigma maps
    sigma_base = calculate_sigma_map(base_dist, config['blur_start'], config['blur_growth_rate'])
    full_blur = apply_spatial_gaussian_blur(img, sigma_base, config['sigma_levels'], config['ksizes'])
    background_blurred = np.where(bg_mask[..., np.newaxis].astype(bool), full_blur, img)

    sigma_obj = calculate_sigma_map(
        base_dist,
        config['blur_start'] * config.get('blur_start_height_multiplier', 1.0),
        config['blur_growth_rate']
    )
    object_blur = apply_spatial_gaussian_blur(img, sigma_obj, config['sigma_levels'], config['ksizes'])
    final = np.where(object_mask[..., np.newaxis].astype(bool), object_blur, background_blurred)


    # Optional color tint for all blurred pixels
    if 'blur_color' in config:
    # 1) get mask from sigma, not pixel-diff
      blur_mask = sigma_base > 0
      color = tuple(config['blur_color'])
      alpha = config.get('blur_color_alpha', 0.3)
      final = apply_color_tint(final, blur_mask, color, alpha)
    cv2.imwrite(config['output_path'], final)
    print(f"Output saved to {config['output_path']}")


if __name__ == '__main__':
    import argparse, json, os # import os

    # Instead of using argparse, define arguments directly or use a configuration file
    class Args:
        def __init__(self, image, depth, type, config, use_synthmap):
            self.image = image
            self.depth = depth
            self.type = type
            self.config = config
            self.use_synthmap = use_synthmap

    # Provide values for the arguments
    args = Args(
        image='gridTest.png',  # Update with your image path
        depth='path/to/your/depth_map.png',  # Update with your depth map if not using --use-heart
        type='person',  # Update with your desired object type
        config='config.json',  # Update with your config file if needed
        use_sythmap=True  # Set to True to use the artificial heart depth map
    )

    # Configuration file handling -- Generates a config file for future use, edit that file or delete and edit this section for changes
    if not os.path.exists(args.config):
        default_config = {
            "model_path": "yolov8n-seg.pt",
            "device": "cpu",
            "class_mapping": {"person": 0},
            "blur_start": 0.2,
            "blur_growth_rate": 0.5,
            "blur_start_height_multiplier": 1.2,
            "sigma_levels": [0, 5, 15, 30],
            "ksizes": [1, 11, 31, 61],
            "output_path": "output_image.png",
            "blur_color": "0,0,255",
            "blur_color_alpha": 0.3
        }
        with open(args.config, 'w') as f:
            json.dump(default_config, f, indent=4)
        print(f"Created default config file: {args.config}")

    cfg = json.load(open(args.config))
    print("Config I’m actually using:\n", cfg)
    # Parse blur color argument (if provided in the configuration file)
    if 'blur_color' in cfg:
        print("Blur found")
        try:
            cfg['blur_color'] = [int(c) for c in cfg['blur_color'].split(',')]
        except Exception:
            raise ValueError("blur_color in config file must be in format B,G,R with integer values")

    main(args.image, args.depth, args.type, cfg, use_synthmap=args.use_synthmap)

NameError: name 'Any' is not defined